In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys, copy
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *

#------ Path towards the required folders
simdir     = 'sim_data_dir_example/3x2v_example' # This is the simulation directory relative to the g0sim
fileprefix = 'gk_tcv_posD_iwl_3x2v_D02' # This is the prefix before the '-*.gkyl'

#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                 # Elongation factor
    delta       = 0.3,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)

simulation.set_data_param( simdir = simdir, fileprefix = fileprefix, species = simulation.species)

simulation.geom_param.load_metric(simulation.data_param.fileprefix)

#----- set up the sources
n_srcOMP=2.4e23
x_srcOMP=0.0
Te_srcOMP=2 * simulation.species['elc'].T0
Ti_srcOMP=2 * simulation.species['ion'].T0
sigma_srcOMP=0.03 * simulation.geom_param.Lx
floor_src=1e-2
def custom_density_src_profile(x,y,z):
    return n_srcOMP * (np.exp(-((x - x_srcOMP) ** 2) / (2.0 * sigma_srcOMP ** 2)) + floor_src)
def custom_temp_src_profile_elc(x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Te_srcOMP; fout[~mask] = Te_srcOMP * 3.0 / 8.0
    return fout  
def custom_temp_src_profile_ion( x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Ti_srcOMP; fout[~mask] = Ti_srcOMP * 3.0 / 8.0
    return fout   

simulation.set_OMPsources(n_srcOMP=n_srcOMP,x_srcOMP=x_srcOMP,
                          Te_srcOMP=Te_srcOMP,Ti_srcOMP=Ti_srcOMP,
                          sigma_srcOMP=sigma_srcOMP,floor_src=floor_src,
                          density_src_profile=custom_density_src_profile,
                          temp_src_profile_elc=custom_temp_src_profile_elc,
                          temp_src_profile_ion=custom_temp_src_profile_ion)

print("Input power:    %g MW"%(simulation.get_input_power()/1e6))
print("Input particle: %g part/s"%(simulation.get_input_particle()))

# Check available frames of the ES potential
tfs = find_available_frames(simulation,'field')

# ---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','minor radius')
simulation.normalize('y','Larmor radius')
simulation.normalize('z','pi')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','Pa')
simulation.normalize('energies','MJ')

# Check available frames of the ES potential
tfs = find_available_frames(simulation,fileprefix+'-field')
print("Time frames available from %g to %g"%(tfs[0],tfs[-1]))

# ---- Show the integrated moment time evolution
time = plot_integrated_moment(simulation,fieldnames=[
    ['ne','ni'],['upare','upari'],['Tpare','Tpari'],['Tperpe','Tperpi']])

In [ ]:
#------- 1D profile relaxation plot
#-- Plane location
cdirection='x'; ccoords=[0.0,0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='ky'; ccoords=[0.01,0.0]
# cdirection='z'; ccoords=[0.01,'avg']
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = ['ne','phi','upare','upari','Te','Ti','pe','pi'],
                        twindow = tfs[-10:], space_time = False,
                       )

In [ ]:
#------- Space time fluctuation plot
cdirection='x'; ccoords=[0.0,0.0]
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = ['phi'],
                        twindow = tfs[:], space_time = True, fluctuation = True,
                       )

In [ ]:
#---------- 2D cut plot
cplane='xz'; ccoord = 0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = tfs[-100:], 
            fieldnames = ['ne'],
            time_average = False,
            fluctuation = True,
            )

In [ ]:
#---------------- Space-time diagrams
#-- Plane location
cdirection='x'; ccoords=[0.0,0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='ky'; ccoords=[0.01,0.0]
# cdirection='z'; ccoords=[0.01,'avg']
#----------------
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = [
                           'ne','phi',
                           'upare','upari',
                           'Te','Ti'],
                       twindow    = tfs[::2], space_time = True
                       )

In [ ]:
#---------- 2D cut plot
#-- choose min and max time frames and cut location
cplane='xy'; ccoord = 0.0
# cplane='xz'; ccoord = 0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = tfs[-1], 
            fieldnames = [
               'ne','phi',
               'upare','upari',
               'Te','Ti'],
            # fieldnames = ['phi'], # field to plot (4 moments if nothing)
            # clim = [80,105],
            # xlim = [0.98,1.02],
            )

In [ ]:
#------ Profile time-average plot at OBMP of the ExB shear of the last 50 frames
cdirection='x'; ccoords=['avg',0.0]
plot_1D(simulation,cdirection,ccoords,
        fieldnames = ['sExBy_x'],
        tfs = tfs[-50:], errorbar = True,
        )

In [ ]:
#------- Flux surface time average radial profile plots
#-- Plane location
figout = []
cdirection='x'; ccoords=['avg','avg']
plot_1D(simulation,cdirection,ccoords,
        fieldnames = [['ne','ni'],['upare','upari'],
                      ['Te','Ti'],['pe','pi'],
                      'qdens','phi',
                      ['ExB_hflux_xe','ExB_hflux_xi'], ['Wtote','Wtoti']
                     ],
        tfs = tfs[-50::5], errorbar = False, figout = figout
        )
# save_figout(figout,'fig_profiles') # this allows you to reload and replot with additional data

#------- Here is an example for reploting and overlaying data
# figdatadict = load_figout('fig_profiles')

# n_plot = figdatadict[0]
# ne_sim = n_plot['curves'][0]
# ne_exp = {}
# ne_exp['label'] =r'$n_e^{exp}$';
# data = np.loadtxt('ne_exp.txt', delimiter=',') 
# ne_exp['xdata'] = data[:,0]/simulation.normalization['xscale']+1;
# ne_exp['ydata'] = data[:,1];

# plt.plot(ne_sim['xdata'],ne_sim['ydata'],label=ne_sim['label'])
# plt.plot(ne_exp['xdata'],ne_exp['ydata'],'ok',label=ne_exp['label'])
# plt.legend()
# plt.ylabel(r'1/m${^3}$')
# plt.xlabel('r/a')
# plt.show()

# T_plot = figdatadict[2]
# Te_sim = T_plot['curves'][0]
# Te_exp = {}
# Te_exp['label'] =r'$T_e^{exp}$';
# data = np.loadtxt('Te_exp.txt', delimiter=',') 
# Te_exp['xdata'] = data[:,0]/simulation.normalization['xscale']+1;
# Te_exp['ydata'] = data[:,1];

# plt.plot(Te_sim['xdata'],Te_sim['ydata'],label=Te_sim['label'])
# plt.plot(Te_exp['xdata'],Te_exp['ydata'],'ok',label=Te_exp['label'])
# plt.legend()
# plt.ylabel('eV')
# plt.xlabel('r/a')
# plt.show()

In [ ]:
#---- Compute and plot volume integral quantities (energies)
plot_volume_integral_vs_t(simulation,
                          [['Wtote','Wtoti','Wtot'],
                           ['Wkine','Wkini'],
                           ['Wflue','Wflui'],
                           ['Wpote','Wpoti']],
                          # fieldnames = [['Wtot']],
                          tfs=tfs[::10],
                          ddt = False, plot_src_input=True,
                          add_GBloss = False)

In [ ]:
#---- Compute particle loss due to fluctuations over time in the simulation
# (this does not depend on the srcGB parameters)
plot_GB_loss(simulation,tfs[:-1:10],losstype='particle',integrate=False)
plot_GB_loss(simulation,tfs[:-1:10],losstype='energy',integrate=False)

In [ ]:
#---------- 2D cut movie
#-- choose min and max time frames and cut location
cplane='xy'; ccoord=0.0
make_2D_movie(simulation, cplane, ccoord, 
            tfs = tfs[::10], 
            fieldnames = ['ne','phi','Te','Ti'],            
            )

In [13]:
#---------- 2D cuts of a frame of the distribution function (long to load)
if False: #set it to true if you have some dist function data
    # Simulation frame
    tf = tfs[-2]
    # Point where each cuts intersect [in Gkeyll units or indices]
    xf = 0.025
    yf = 0.0
    zf = 0.0
    vf = 0.0
    mf = 0.0
    
    for spec in [elc,ion]:
        #---- Load the distf
        fname = simulation.get_filename('f'+spec.nshort,tf)
        data = pg.data.GData(fname)
        dg = pg.data.GInterpModal(data,1,'ms')
        dg.interpolate(0,overwrite=True)

        #---------------- 2D vpar x plots
        cut = copy.copy(data)
        pg.data.select(cut, z1=yf, z2=zf, z4=mf, overwrite=True)
        pg.output.plot(cut, yscale=1./spec.vt,
                    title=r"y=%2.2f, z=%2.2f, $\mu$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                    contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                    xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$v_\parallel/v_{th%s}$'%spec.nshort,
                    clabel=r'$f_{%s}$'%spec.nshort);

        #---------------- 2D mu x plots
        cut = copy.copy(data)
        pg.data.select(cut, z1=yf, z2=zf, z3=vf, overwrite=True)
        pg.output.plot(cut, yscale=1./spec.mu0,
                    title=r"y=%2.2f, z=%2.2f, $v_\parallel$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                    contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                    xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$\mu B/T_{%s}$'%spec.nshort,
                    clabel=r'$f_{%s}$'%spec.nshort);

        #---------------- 2D vpar mu plots
        cut = copy.copy(data)
        pg.data.select(cut, z0=xf, z1=yf, z2=zf, overwrite=True)
        pg.output.plot(cut, yscale=1./spec.mu0,
                    title=r"x=%2.2f, y=%2.2f, z=%2.2f, t=%2.2e (s)"%(xf,yf,zf,data.ctx['time']),
                    contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                    xlabel=r'$v_\parallel/v_{th%s}$'%spec.nshort,ylabel=r'$\mu B/T_{%s}$'%spec.nshort,
                    clabel=r'$f_{%s}$'%spec.nshort);